In [1]:
%env CUDA_VISIBLE_DEVICES=4

import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

env: CUDA_VISIBLE_DEVICES=4
INFO 04-29 13:01:49 [__init__.py:239] Automatically detected platform cuda.


In [2]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
# %env CUDA_VISIBLE_DEVICES=0

Using GPU: cuda:4


In [3]:
with open(os.path.abspath('../openai_key'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [4]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

In [5]:
def count_words(text):
    """
    Counts the number of words in the given text.

    Args:
        text (str): Input text.

    Returns:
        int: Number of words in the text.
    """
    if text!=None:
        words = text.split()
        return len(words)
    else:
        return 0


In [6]:
personas = [
  # Elementary
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary school student with a Narrative learning style, I absorb new concepts best when they’re told as engaging mini-stories. "
      "In dialogue, I ask for short anecdotes that turn any abstract idea into a vivid tale with characters, a clear sequence, and an emotional hook. "
      "Stories help me remember causal links and keep details alive in my mind."
    )
  },
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary school student with a Kinesthetic learning style, I understand ideas by imagining myself performing them. "
      "In conversation, I ask you to guide me through a pretend play-through—verbally walking me step by step as if I’m enacting a simple experiment or physical process. "
      "This imagined movement helps me anchor concepts in ‘muscle memory’ even though we’re only talking."
    )
  },
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary school student with a Naturalistic learning style, I connect best when content is tied to the natural world through vivid imagery. "
      "In dialogue, I ask you to compare topics—like atomic structure—to things I observe outdoors, such as tree rings or bird migrations. "
      "These verbal nature metaphors make new information feel familiar and alive."
    )
  },
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary school student with an Experiential learning style, I learn by mentally simulating real-world tasks. "
      "In conversation, I ask you to walk me through building or testing something—describing each step as if I’m doing it. "
      "That imagined ‘doing’ makes concepts concrete, even though we remain in chat."
    )
  },
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary school student with a Creative-Divergent learning style, I thrive on brainstorming multiple possibilities. "
      "In dialogue, I propose ‘what if’ scenarios—like alternative endings or playful twists on a concept—and talk through each idea. "
      "Verbal brainstorming reveals fresh patterns and sparks my imagination."
    )
  },

  # Middle
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school student with a Visual-Spatial learning style, I think in mental images and diagrams. "
      "In conversation, I ask you to ‘paint’ word-pictures—step-by-step descriptions of scenes or flows—so I can build a clear mental map. "
      "That verbal imagery helps me organize information spatially in my mind."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school student with an Auditory learning style, I internalize knowledge through sound and speech. "
      "In dialogue, I ask you to restate key points in different rhythms or tones, and I repeat them back to reinforce my memory. "
      "Hearing and echoing concepts in conversation makes them stick."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school student with a Logical-Mathematical learning style, I seek numerical patterns and rule-based reasoning. "
      "In dialogue, I pose ‘what-if’ questions—‘If X doubles, what changes?’—and we talk through each scenario using simple calculations. "
      "Quantitative hypotheticals build my systematic understanding."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school student with an Analytical-Argument learning style, I dissect arguments and causal chains. "
      "In conversation, I ask targeted ‘why’ and ‘how’ questions about each step, construct mini flow-charts aloud, and verify the logic with you. "
      "This structured debate hones my precision in reasoning."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school student with a Verbal-Linguistic learning style, I learn through rich language and writing. "
      "In dialogue, I request carefully worded definitions, paraphrase ideas in my own words, and craft mnemonic rhymes on the spot. "
      "Talking through ideas in text-like sentences and playing with words helps me remember precisely."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school student with a Technology-Enhanced learning style, I thrive on conversational simulations of digital tools. "
      "In dialogue, I ask you to describe how a virtual model might respond as we adjust parameters, or to role-play a flashcard quiz verbally. "
      "These imagined tech interactions keep me engaged without leaving our chat."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school student with a Mnemonic learning style, I anchor facts with memory aids. "
      "In dialogue, I ask for catchy acronyms, rhymes, or vivid mental images—then recite them back. "
      "That verbal encoding makes complex lists or steps easy to retrieve."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school student with an Emotional learning style, I connect through feelings and empathy. "
      "In conversation, I ask you to frame concepts in human-centered narratives that highlight emotional stakes. "
      "These emotionally rich verbal stories make ideas memorable and meaningful."
    )
  },

  # High School
  {
    "grade_level": "high school",
    "description": (
      "As a high school student with a Collaborative learning style, I excel in multi-voice discussions. "
      "In dialogue, I invite hypothetical peers into our chat—debating viewpoints, role-playing characters, or comparing interpretations. "
      "That social exchange refines my understanding."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school student with an Interpersonal learning style, I flourish in one-on-one exchanges. "
      "In conversation, I engage deeply with a single partner—asking questions, providing feedback, and co-constructing ideas through back-and-forth talk."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school student with a Reflective learning style, I pause and summarize before responding. "
      "In dialogue, I restate points in my own words, journal key ideas mentally, and then ask precise follow-ups. "
      "This verbal reflection clarifies gaps and deepens comprehension."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school student with a Metaphorical learning style, I anchor concepts in analogies. "
      "In dialogue, I ask you to compare subjects to familiar scenarios—‘It’s like X because…’—and we talk through how well the metaphor holds. "
      "Testing analogies verbally helps me translate abstract ideas into relatable terms."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school student with an Intrapersonal learning style, I connect content to my own values. "
      "In dialogue, I ask how topics relate to my goals or experiences and share personal reflections aloud. "
      "That self-referential talk makes learning relevant and motivating."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school student with a Problem-Based learning style, I tackle hypothetical real-world scenarios in talk. "
      "In dialogue, I propose case studies—like designing a sustainable system—and we walk through each decision together. "
      "Verbal scenario-based reasoning shows me practical applications of theory."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school student with a Trial-and-Error learning style, I learn by mentally testing ideas. "
      "In dialogue, I suggest imagined experiments—‘Let’s tweak this variable and see what happens’—and we discuss the outcomes. "
      "Using mistakes as discussion points builds discovery-based understanding."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school student with a Conceptual learning style, I focus on verbal mapping of frameworks. "
      "In dialogue, I request thematic overviews—described step by step—and we discuss how each piece fits into the big picture. "
      "Building mental models in talk deepens my flexible understanding."
    )
  },

  # College
  {
    "grade_level": "college",
    "description": (
      "As a college student with a Theoretical learning style, I probe abstract frameworks in conversation. "
      "In dialogue, I challenge you to trace ideas back to their assumptions, compare theoretical models, and debate implications. "
      "This verbal inquiry drives deep synthesis."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college student with a Research-Oriented learning style, I learn by interrogating studies in chat. "
      "In conversation, I ask for summaries of current research, discuss methods and controls, and role-play peer-review feedback. "
      "Critically evaluating evidence through talk builds an evidence-based grasp."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college student with an Integrative learning style, I weave ideas together verbally. "
      "In conversation, I ask for cross-topic syntheses—connecting historical, artistic, and scientific themes—and discuss their intersections step by step. "
      "This systems-level perspective helps me approach complex questions creatively."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college student with a Structured learning style, I excel on verbal outlines and modules. "
      "In dialogue, I ask for hierarchical breakdowns—numbered lists, staged explanations, and schematic overviews—before diving into details."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college student with a Solitary learning style, I prefer self-guided dialog prompts. "
      "In our conversation, I request personalized questions and silent think-time before sharing my conclusions, using chat as a safe space for independent reflection."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college student with an Adaptive learning style, I shift strategies based on what works. "
      "In dialogue, I monitor which verbal approaches—stories, logic puzzles, analogies—help me most and ask to switch accordingly. "
      "This dynamic, metacognitive talk ensures I absorb concepts through the most effective modality."
    )
  }
]


In [7]:
import json
import shutil
import os

config_role = {
    "agent1_role": "Teacher",
    "agent2_role": "Student",
    "agent1_prompt": "You are a teacher whose goal is to guide a student through learning about %SUBJECT%. You have a preferred way to teach the student. The student is in %ROLE% so make sure to teach them at their level. ",
    "agent2_prompt": "You are a student in %ROLE% in conversation with a teacher who will teach you %SUBJECT%. You like to learn in the following way:\n%SPEAKER_BACKSTORY%.\nMake sure to not only ask questions but also demonstrate your knowledge.",
    'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said.\n",
    'eval_prompts': {
    'prompt_consistency': """You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given background. Answer YES if the line directly contradicts any part of the background—this includes contradictions in facts, behavior, beliefs, emotional stance, or conversational strategy. Answer NO if the line introduces new details, but those details are **plausibly aligned** with the background. Be strict and literal in your evaluation: if the line violates the persona’s stated traits (e.g., avoids vulnerability, keeps answers short, avoids discussing personal topics), mark it as a contradiction even if the violation is subtle. Background for %SPEAKER_ROLE%: %SPEAKER_BACKSTORY% Line spoken by %SPEAKER_ROLE%: %SPEAKER_LINE%  Answer YES if the line contradicts the background, and NO if it does not. Then, explain your answer in one sentence. Be precise and avoid vague justification."""}}

os.makedirs("education", exist_ok=True)
with open("education/config_education.json", "w", encoding="utf-8") as f:
    json.dump(config_role, f, indent=4)

In [8]:
with open('education/personas_education_master.json', 'r') as f:
    conversation_prompts = json.load(f)
conversation_prompts[0]['background_info'].keys()

dict_keys(['topic', 'student_prefrences', 'teacher_prefrences', 'student_reactions', 'teacher_reactions'])

In [9]:
topic_list = []
for convo_prompt in conversation_prompts:
    topic_prompt = convo_prompt["background_info"]["topic"]
    topic_list.append(topic_prompt)

In [10]:
llms = ["Llama-3.1-8B-Instruct", "gpt-4o-mini", "Qwen2.5-3B-Instruct", "Llama-3.1-8B", "Mistral-7B-Instruct", "Llama-3.1-70B", "Llama-3.1-70B-Instruct", "phi-3.5-mini-instruct"]
        
config_llm = {'agent1_model': 'Llama-3.1-8B-Instruct',
             'agent2_model': 'Llama-3.1-8B-Instruct',
             'eval_model': 'Llama-3.1-70B-Instruct',
             'iterations': 10,
             'verbose': False,
             'write': True,
             'convo_length_limit': 10,
             'max_tokens': 256,
             'gpus': 1,
             'seed': 0,
             'task_name': 'Education',
             'model_dir': "/home/marwa/models/"}

with open("education/Llama-3.1-8B-Instruct.json", "w", encoding="utf-8") as f:
    json.dump(config_llm, f, indent=4)

In [11]:
import re

def clean_role_prefix(response, expected_role):
    """
    Removes repeated instances of the expected_role prefix at the start (e.g., 'Therapist: Therapist:'),
    and ensures the response begins with a single correct expected_role prefix.
    """
    pattern = rf"^(({re.escape(expected_role)}):\s*)+"
    cleaned = re.sub(pattern, '', response.strip(), flags=re.IGNORECASE)
    return cleaned
    
def is_role_confused(response, other_role):
    """
    Checks if the output starts with the wrong speaker tag.
    """
    if other_role + ":" in response:
        return True
    else: 
        return False

def generate_response(agent_model, expected_role, other_role, config_llm, prompt, max_retries=3):
    role_confused = True
    while(role_confused):
        response = completion_create(agent_model, config_llm, prompt)
        print("Expected Role", expected_role)
        role_confused = is_role_confused(response, other_role)
        if not is_role_confused(response, other_role):
            return clean_role_prefix(response, expected_role)
            
    return clean_role_prefix(response, expected_role)

def generate_conversation(config_llm, p1, p2, p1_name, p2_name, subject, role, pturn=1):
    stats['P1'] = p1
    stats['P2'] = p2
    stats["topic"] = subject
    stats["grade"] = role
    stats['pturn'] = pturn
    round_num = 0
    while round_num < config_llm['convo_length_limit']:
        conversation = ("".join([turn[1] if isinstance(turn, tuple) else turn for turn in stats["conversation"]]) if len(stats["conversation"]) != 0 else "You are starting the conversation.\n")

        if pturn == 1:
            prompt = config_role["agent1_prompt"]
            pturn = 2
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the student so far is below:\nConversation:\n%CONVERSATION%"
                
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."

            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the student. Remember you are the teacher. "
                
            prompt += config_role["reminder_prompt"]
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_role["agent1_role"]) \
                   .replace("%LISTENER_ROLE%", config_role["agent2_role"]) \
                    .replace("%ROLE%", role) \
                   .replace("%SUBJECT%", subject) \
                   .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent1_model'], config_role["agent1_role"], config_role["agent2_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_role["agent1_role"]}: " + response + "\n"))
        
        else:
            prompt = config_role["agent2_prompt"]
            pturn = 1    
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the teacher so far is below:\nConversation:\n%CONVERSATION%"
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."
            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the teacher. Remember you are the student. "

            prompt += config_role["reminder_prompt"]
            
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_role["agent2_role"]) \
               .replace("%LISTENER_ROLE%", config_role["agent1_role"]) \
               .replace("%SPEAKER_BACKSTORY%", p2) \
                .replace("%ROLE%", role) \
               .replace("%SUBJECT%", subject) \
               .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent2_model'], config_role["agent2_role"], config_role["agent1_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_role["agent2_role"]}: " + response + "\n"))
            
        round_num += 1

    stats["rounds"] = round_num
    if config_llm['verbose']:
        print(stats["conversation"])
    return stats.copy()

def reset_stats():
    stats_template = {
        "task_name": config_llm['task_name'],
        "topic": "",
        "grade": "",
        "P1": "",
        "P2": "",
        "conversation": [],
        "pturn": 0, # beginning person (1 or 2)
        "index": -1,
        "timestamp": "",
        "rounds": 0,
        'conversation_only': True
    }
    for key, value in stats_template.items():
        stats[key] = value

In [12]:
import os
import random
from datetime import datetime
import utils
utils.config = config_llm

current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"education/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# Generate unique random number for filename
def generate_unique_file_number(output_dir, prefix, seed, extension=".json"):
    while True:
        rand_num = random.randint(0, 1000)
        filename = f"{prefix}_{seed}_{rand_num}{extension}"
        filepath = os.path.join(output_dir, filename)
        if not os.path.exists(filepath):
            return rand_num

unique_num = generate_unique_file_number(
    output_dir,
    config_llm['agent1_model'],
    config_llm['seed']
)

# File to write output to
write_file = os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_{unique_num}.json")

In [13]:
import random

# 1. Build a “persona pool” of length 100
pool = personas * (100 // len(personas)) \
     + random.sample(personas, 100 % len(personas))
random.shuffle(pool)

# 2. Sample 100 topics *with* replacement
topic_choices = random.choices(topic_list, k=100)

# 3. Zip them into 100 pairs
persona_final = list(zip(topic_choices, pool))
assert len(persona_final) == 100

In [14]:
with open("education/config_education_personas.json", "w", encoding="utf-8") as f:
    json.dump(persona_final, f, indent=4)

In [15]:
eval_prompts = {
    "strategy_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, provide 1 sentence explaining your reasoning based on the strategy, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s strategy is described as follows:\n%SPEAKER_STRATEGY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reasoning followed by YES or NO.\n\n",
    "background_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. Provide 1 sentence explaining your reasoning based on the background, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reason reasoning followed by YES or NO.\n\n",
    "combined_prompt_consistency": "%SCENARIO_DESC% Evaluate the intention behind the following line spoken by %SPEAKER_ROLE% and determine whether it contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, answer YES if the line contradicts the given background of %SPEAKER_ROLE% or the intention does not align with the provided background, and answer NO if it does align with the provided background or the intention aligns with the background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as 1 sentence explaining your reasoning based on the background and the interpreted intention, followed by YES or NO.\n\n",
    
    "forwards_combined_prompt_consistency": "%SCENARIO_DESC% Evaluate the intention behind the following line spoken by %SPEAKER_ROLE% and determine whether it contradicts their background. Answer YES if the line contradicts the given background of %SPEAKER_ROLE% or the intention does not align with the provided background, and answer NO if it does align with the provided background or the intention aligns with the background of %SPEAKER_ROLE%, then describe the interpreted intention of the statement and whether or not it aligns with the given background of %SPEAKER_ROLE% within 1 sentence. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as YES or NO first, followed by 1 sentence explaining your reasoning based on the background and the interpreted intention.\n\n",
    
    "index_consistency":"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, first determine if there is a CLEAR conflict or inconsistency between the line and any line within the conversation history spoken by %SPEAKER_ROLE%. IF there is a conflict, provide a sentence of reasoning followed by a list of indices of lines in the conversation history that have a clear conflict with the current line. Otherwise, provide a sentence of reasoning followed by an empty list. ONLY INCLUDE INDICES OF LINES THAT CORRESPOND TO %SPEAKER_ROLE%. The conversation up to this point is as follows: %CONVERSATION%. %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your reasoning as 1 sentence, followed by a list of indices of conflicting lines from the conversation history formatted like a Python list in the following format: [index1, index2, index3, ...].\n\n",
    
    "pairwise_consistency":"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, answer YES if the line directly contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%. %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n %LISTENER_ROLE% spoke the following line: \n%LISTENER_LINE%\n\n Answer YES if the line spoken by %SPEAKER_ROLE% contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%, followed by 1 sentence of reasoning.\n\n",

    "backstory_test": "Based on the following background, generate a new fact-based multiple choice question with 5 choices addressed directly IN SECOND PERSON, along with its correct answer. Preface the question with 'Question:' and the answer with 'Answer:'.\n%SPEAKER_BACKSTORY%\n%PREVIOUS_QUESTIONS%",
    "answer_backstory": "You are %SPEAKER_ROLE%, and you are having a conversation with %LISTENER_ROLE%. Your background is:\n%SPEAKER_BACKSTORY%\n So far, the conversation is as below:\n%CONVERSATION%\n\n Based on your conversation above so far, answer the following multiple choice question.\n%BACKSTORY_QUESTION%\n",
    "grade_backstory": "As part of grading a test, determine whether the given answer %GIVEN_ANSWER% matches the following correct answer. Respond with either YES or NO.\nCorrect Answer: %CORRECT_ANSWER%\n"
}

In [ ]:
import consistency_eval
consistency_eval.prompts = config_role
consistency_eval.config = config_llm
consistency_eval.eval_prompts = eval_prompts
index_offset = load_stats_file(write_file)
conversations = []    
lengths = [10, 20, 40, 60]
for i in range(1):
    for topic, persona_item in persona_final[77:]:
        background = persona_item["description"]
        grade = persona_item["grade_level"]
        for convo_length in lengths:
            config_llm['convo_length_limit'] = convo_length
            reset_stats()
            conversation = generate_conversation(
                config_llm,
                "", 
                background, 
                "Teacher", 
                "Student", 
                topic, 
                grade, 
                pturn=1
            )
            conversation_eval = consistency_eval.eval_prompt_consistency(conversation, both_agents=False)
            conversation_eval = consistency_eval.eval_index_consistency(conversation_eval, both_agents=False)

            print(conversation_eval)
            conversations.append(conversation_eval)
            stats['index'] = index_offset
            stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            write_stats(write_file)
            index_offset += 1


written!!
INFO 04-29 13:01:59 [config.py:600] This model supports multiple tasks: {'generate', 'classify', 'embed', 'reward', 'score'}. Defaulting to 'generate'.
INFO 04-29 13:01:59 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 04-29 13:02:01 [utils.py:2273] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 04-29 13:02:05 [__init__.py:239] Automatically detected platform cuda.
INFO 04-29 13:02:08 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  6.75it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.57it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.05it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  1.88it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.10it/s]



INFO 04-29 13:02:12 [loader.py:447] Loading weights took 1.99 seconds
INFO 04-29 13:02:12 [gpu_model_runner.py:1273] Model loading took 14.9889 GiB and 3.128088 seconds
INFO 04-29 13:02:20 [backends.py:416] Using cache directory: /home/marwa/.cache/vllm/torch_compile_cache/8b4ebbe309/rank_0_0 for vLLM's torch.compile
INFO 04-29 13:02:20 [backends.py:426] Dynamo bytecode transform time: 7.82 s
INFO 04-29 13:02:21 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-29 13:02:27 [monitor.py:33] torch.compile takes 7.82 s in total
INFO 04-29 13:02:28 [kv_cache_utils.py:578] GPU KV cache size: 295,152 tokens
INFO 04-29 13:02:28 [kv_cache_utils.py:581] Maximum concurrency for 131,072 tokens per request: 2.25x
INFO 04-29 13:02:47 [gpu_model_runner.py:1608] Graph capturing finished in 19 secs, took 0.53 GiB
INFO 04-29 13:02:47 [core.py:162] init engine (profile, create kv cache, warmup model) took 35.01 seconds


Processed prompts: 100%|███████████████████████████████████| 1/1 [00:00<00:00,  1.47it/s, est. speed input: 150.56 toks/s, output: 78.23 toks/s]


Expected Role Teacher


...

Processed prompts: 100%|███████████████████████████████████| 1/1 [00:00<00:00,  1.09it/s, est. speed input: 511.41 toks/s, output: 81.61 toks/s]


Expected Role Teacher


Processed prompts: 100%|███████████████████████████████████| 1/1 [00:00<00:00,  1.36it/s, est. speed input: 818.33 toks/s, output: 81.83 toks/s]


Expected Role Student


Processed prompts:   0%|                                              | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
write_stats(write_file)

In [ ]:
len(conversations)

In [ ]:
# with open("education/exp/04.28.25/Mistral-7B-Instruct_0_751.json", "w", encoding="utf-8") as f:
#     json.dump(conversations, f, indent=4)

In [ ]:
conversations

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch

# 1. Load dialogues and the single consistency score from JSON files for each model
model_files = {
    "ModelA": "modelA_conversations.json",
    "ModelB": "modelB_conversations.json",
    "ModelC": "modelC_conversations.json"
}

dfs = []
for model_name, filepath in model_files.items():
    with open(filepath, 'r') as f:
        data = json.load(f)
    records = []
    for entry in data:
        # Adjust field name if needed to match your JSON key
        score = entry.get("P2_prompt_consistency_score", entry.get("consistency_score"))
        records.append({
            "persona": entry["P2"],
            "consistency_score": score,
            "model": model_name
        })
    dfs.append(pd.DataFrame(records))

df = pd.concat(dfs, ignore_index=True)

# 2. Merge with persona metadata to attach grade levels
# Assumes a personas.json file: [{"grade_level":..., "description":...}, ...]
with open("personas.json", "r") as f:
    personas_meta = json.load(f)
meta_df = pd.DataFrame(personas_meta).rename(columns={"description": "persona"})
df = df.merge(meta_df, on="persona", how="left")

# 3. Boxplot: consistency distribution by model
plt.figure()
df.boxplot(column="consistency_score", by="model")
plt.title("Consistency Score Distribution by Model")
plt.suptitle("")
plt.ylabel("Consistency Score")
plt.xlabel("Model")
plt.show()

# 4. Heatmap: mean consistency per persona and model
pivot = df.pivot_table(
    index="persona", columns="model", values="consistency_score", aggfunc="mean"
)
plt.figure()
plt.imshow(pivot.values, aspect='auto', interpolation='none')
plt.xticks(ticks=np.arange(len(pivot.columns)), labels=pivot.columns, rotation=45, ha='right')
plt.yticks(ticks=np.arange(len(pivot.index)), labels=pivot.index)
plt.colorbar(label="Mean Consistency")
plt.title("Mean Consistency Heatmap (Persona × Model)")
plt.tight_layout()
plt.show()

# 5. Bar chart: average consistency by grade level for each model
grade_means = df.groupby(["grade_level", "model"])["consistency_score"].mean().unstack()
plt.figure()
grade_means.plot(kind="bar")
plt.title("Average Consistency by Grade Level and Model")
plt.ylabel("Mean Consistency Score")
plt.xlabel("Grade Level")
plt.legend(title="Model")
plt.tight_layout()
plt.show()

# 6. Dendrogram: cluster personas by their consistency profile across models
# Create matrix: rows=persona, cols=models
profile = df.pivot_table(
    index="persona", columns="model", values="consistency_score", aggfunc="mean"
)
Z = sch.linkage(profile.values, method='average', metric='euclidean')
plt.figure(figsize=(10, 5))
sch.dendrogram(Z, labels=profile.index, leaf_rotation=90)
plt.title("Dendrogram of Personas by Consistency Profile")
plt.ylabel("Distance")
plt.tight_layout()
plt.show()
